## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/SMSSpamCollection"
file_type = "parquet"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "SMSSpamCollection"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `SMSSpamCollection`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "SMSSpamCollection"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
import pyspark

In [0]:
data=spark.read.csv("/FileStore/tables/SMSSpamCollection",inferSchema=True,sep='\t')

In [0]:
data.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

In [0]:
data=data.withColumnRenamed("_c0","class").withColumnRenamed("_c1","text")

In [0]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import length

In [0]:
data=data.withColumn("length",length(data["text"]))

In [0]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [0]:
data.groupBy("class").mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer
tokenizer=Tokenizer(inputCol="text",outputCol="token_text")
stopremove=StopWordsRemover(inputCol="token_text",outputCol="stop_tokens")
count_vec=CountVectorizer(inputCol="stop_tokens",outputCol="c_vec")
idf=IDF(inputCol="c_vec",outputCol="tf_idf")
ham_to_spam_number=StringIndexer(inputCol="class",outputCol="label")

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [0]:
vector_assebmler=VectorAssembler(inputCols=['tf_idf','length'], outputCol="features")

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb=NaiveBayes()

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_pipeline=Pipeline(stages=[ham_to_spam_number,tokenizer,stopremove,count_vec,idf,vector_assebmler])

In [0]:
clean=data_pipeline.fit(data)

In [0]:
clean_data=clean.transform(data)

In [0]:
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
class| text|length|label| token_text| stop_tokens| c_vec| tf_idf| features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 ham|Go until jurong p...| 111| 0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
 ham|Ok lar... Joking ...| 29| 0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
 spam|Free entry in 2 a...| 155| 1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
 ham|U dun say so earl...| 49| 0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...|(13424,[0,70,80,1...|
 ham|Nah I don't think...| 61| 0.0|[nah, i, don't, t...|[nah, think, goes...|(13423,[36,134,31...|(13423,[36,134,31...|(13424,[36,134,31...|
 spam|FreeMsg Hey there...| 147| 1.0|[freemsg, hey, th...|[freemsg, hey, da...|(13423,[10,60,139...|(13423,[10,60,139...|(13424,[10,60,139...|
 ham|Even my brother i...| 77| 0.0|[even, my, brothe...|[even, brother, l...|(13423,[10,53,103...|(13423,[10,53,103...|(13424,[10,53,103...|
 ham|As per your reque...| 160| 0.0|[as, per, your, r...|[per, request, 'm...|(13423,[125,184,4...|(13423,[125,184,4...|(13424,[125,184,4...|
 spam|WINNER!! As a val...| 157| 1.0|[winner!!, as, a,...|[winner!!, valued...|(13423,[1,47,118,...|(13423,[1,47,118,...|(13424,[1,47,118,...|
 spam|Had your mobile 1...| 154| 1.0|[had, your, mobil...|[mobile, 11, mont...|(13423,[0,1,13,27...|(13423,[0,1,13,27...|(13424,[0,1,13,27...|
 ham|I'm gonna be home...| 109| 0.0|[i'm, gonna, be, ...|[gonna, home, soo...|(13423,[18,43,120...|(13423,[18,43,120...|(13424,[18,43,120...|
 spam|SIX chances to wi...| 136| 1.0|[six, chances, to...|[six, chances, wi...|(13423,[8,17,37,8...|(13423,[8,17,37,8...|(13424,[8,17,37,8...|
 spam|URGENT! You have ...| 155| 1.0|[urgent!, you, ha...|[urgent!, won, 1,...|(13423,[13,30,47,...|(13423,[13,30,47,...|(13424,[13,30,47,...|
 ham|I've been searchi...| 196| 0.0|[i've, been, sear...|[searching, right...|(13423,[39,96,217...|(13423,[39,96,217...|(13424,[39,96,217...|
 ham|I HAVE A DATE ON ...| 35| 0.0|[i, have, a, date...|[date, sunday, wi...|(13423,[552,1697,...|(13423,[552,1697,...|(13424,[552,1697,...|
 spam|XXXMobileMovieClu...| 149| 1.0|[xxxmobilemoviecl...|[xxxmobilemoviecl...|(13423,[30,109,11...|(13423,[30,109,11...|(13424,[30,109,11...|
 ham|Oh k...i'm watchi...| 26| 0.0|[oh, k...i'm, wat...|[oh, k...i'm, wat...|(13423,[82,214,47...|(13423,[82,214,47...|(13424,[82,214,47...|
 ham|Eh u remember how...| 81| 0.0|[eh, u, remember,...|[eh, u, remember,...|(13423,[0,2,49,13...|(13423,[0,2,49,13...|(13424,[0,2,49,13...|
 ham|Fine if thats th...| 56| 0.0|[fine, if, thats...|[fine, thats, wa...|(13423,[0,74,105,...|(13423,[0,74,105,...|(13424,[0,74,105,...|
 spam|England v Macedon...| 155| 1.0|[england, v, mace...|[england, v, mace...|(13423,[4,30,33,5...|(13423,[4,30,33,5...|(13424,[4,30,33,5...|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
clean_data = clean_data.select(['label','features'])

In [0]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [0]:
train_data,test_data=clean_data.randomSplit([0.7,0.3])

In [0]:
spam_detector=nb.fit(train_data)

In [0]:
test_results = spam_detector.transform(test_data)

In [0]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,13,...|[-609.50608169054...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,2,41,...|[-1062.9439316383...|[1.0,3.5252145794...| 0.0|
 0.0|(13424,[0,1,5,20,...|[-804.69088667306...|[1.0,5.9304628060...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-885.82641322392...|[1.0,6.4941790761...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-541.04888986300...|[1.0,1.1085705255...| 0.0|
 0.0|(13424,[0,1,11,32...|[-892.21352203561...|[1.0,8.0489214584...| 0.0|
 0.0|(13424,[0,1,14,18...|[-1376.9432531656...|[1.0,1.9039747188...| 0.0|
 0.0|(13424,[0,1,14,31...|[-219.33992585080...|[1.0,2.8564493360...| 0.0|
 0.0|(13424,[0,1,17,19...|[-805.01029559985...|[1.0,2.3554550988...| 0.0|
 0.0|(13424,[0,1,21,27...|[-772.43885415607...|[1.0,6.1010950518...| 0.0|
 0.0|(13424,[0,1,23,63...|[-1292.7761964797...|[1.0,1.2523894518...| 0.0|
 0.0|(13424,[0,1,46,17...|[-1135.8045002935...|[1.27431109790873...| 1.0|
 0.0|(13424,[0,1,146,1...|[-250.44134813453...|[0.89091762360380...| 0.0|
 0.0|(13424,[0,1,416,6...|[-301.79993175300...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,498,5...|[-322.30707671334...|[0.99999999999419...| 0.0|
 0.0|(13424,[0,1,874,1...|[-96.148925352002...|[0.99999997035129...| 0.0|
 0.0|(13424,[0,2,3,5,6...|[-2560.7923742159...|[1.0,9.9459786666...| 0.0|
 0.0|(13424,[0,2,3,5,3...|[-504.07550346633...|[1.0,2.4748111667...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3779.7992424759...|[1.0,5.3301012087...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3779.7992424759...|[1.0,5.3301012087...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval=MulticlassClassificationEvaluator(metricName="accuracy")

In [0]:
acc=acc_eval.evaluate(test_results)

In [0]:
acc

Out[40]: 0.9208211143695014